<a href="https://colab.research.google.com/github/SCS-Technology-and-Innovation/DACS/blob/main/DTDA/compromise.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [60]:
n = 200 # how many alternatives to generate

import pandas as pd
from random import randint

# assign a price to each car in CAD
price = pd.Series([ randint(3500, 35000) for car in range(n) ], name = 'Price')

# assign a milage 
milage = pd.Series([ randint(200, 150000) for car in range(n) ], name = 'Milage')

# assign how many miles per gallon the car runs on average
mpg = pd.Series([ randint(30, 70) for car in range(n) ], name = 'FuelEfficiency')

# assign carbon dioxide emissions per gallon in grams
co2 = pd.Series([ randint(8000, 9000) for car in range(n) ], name = 'Emissions')

# subjective coolness on scale from 1 (lame) to 5 (neat)
cool = pd.Series([ randint(1, 5) for car in range(n) ], name = 'Rating')

In [67]:
import pandas as pd

attributes = [ price, milage, mpg, co2, cool ]
cars = pd.concat(attributes, axis = 1)
cars.head()

,Price,Milage,FuelEfficiency,Emissions,Rating
0,25946,33892,60,8723,4
1,16276,137613,53,8185,1
2,11052,40698,66,8279,3
3,13295,112111,57,8552,1
4,7945,140532,52,8147,1


In [68]:
cheapest = cars['Price'].idxmin()
cars.loc[cheapest]

Price              3608
Milage            12377
FuelEfficiency       69
Emissions          8129
Rating                1
Name: 62, dtype: int64

In [69]:
cars.loc[ cars['Rating'] == 5 ]

,Price,Milage,FuelEfficiency,Emissions,Rating
13,7233,24278,41,8267,5
16,25057,59212,60,8928,5
20,5320,30628,65,8687,5
28,13063,99591,37,8467,5
31,7181,19446,57,8081,5
35,29361,18505,46,8598,5
39,31851,18656,63,8184,5
46,9425,76296,47,8043,5
48,4548,5197,57,8181,5
55,33874,90715,50,8070,5


In [70]:
cars[ (cars['Rating'] == 5) & (cars['Price'] < 10000) ]

,Price,Milage,FuelEfficiency,Emissions,Rating
13,7233,24278,41,8267,5
20,5320,30628,65,8687,5
31,7181,19446,57,8081,5
46,9425,76296,47,8043,5
48,4548,5197,57,8181,5
71,5333,127989,35,8966,5
77,4330,10807,38,8391,5
123,7261,117068,32,8707,5
134,5679,3074,54,8030,5
161,5313,111591,34,8991,5


In [71]:
ideal = { 
  'Price' : 1, # minimize
  'Milage' : 1, # also minimize
  'FuelEfficiency' : - 1, # maximize (minimize the negative)
  'Emissions' : 1, # minimize again
  'Rating' : -1 # maximize
} 

multipliers = [ ideal[c] for c in cars.columns ]
multipliers

[1, 1, -1, 1, -1]

In [72]:
def dominates(challenger, challenged): # smaller is better
  if all(v <= w for v, w in zip(challenger, challenged)): # if all aspects are at least as good
    return any( v < w for v, w in zip(challenger, challenged)) # and at least one is actually better
  return False


x = [ 1, 2, 3 ]
y = [ 0, 1, 2 ]
z = [ 1, 4, 3 ]

print('x vs. y', dominates(x, y))
print('y vs. x', dominates(y, x))
print('x vs. z', dominates(x, z))
print('z vs. x', dominates(z, x))
print('z vs. y', dominates(z, y))
print('y vs. z', dominates(y, z))

x vs. y False
y vs. x True
x vs. z True
z vs. x False
z vs. y False
y vs. z True


In [73]:
nondominated = set()

for i, data in cars.iterrows():
  car = [ m * v for m, v in zip(multipliers, data) ]
  discard = False # assume it to be invincible
  for j in range(i + 1, n): # check all the other cars
    otherData = cars.iloc[j] # get their data
    alt = [ m * v for m, v in zip(multipliers, otherData) ]
    if dominates(alt, car): # challenge the car with the alternative
      discard = True # we will not want to consider the loser
      break # no need to check further, it was already beaten
  if not discard: # nothing beat it
    nondominated.add(i) # remember the row

len(nondominated)

57

In [74]:
keep = sorted(list(nondominated))
candidates = cars.iloc[keep]
candidates

,Price,Milage,FuelEfficiency,Emissions,Rating
2,11052,40698,66,8279,3
10,7299,90345,69,8680,4
12,9329,29409,66,8364,4
14,5484,41408,37,8164,2
20,5320,30628,65,8687,5
31,7181,19446,57,8081,5
39,31851,18656,63,8184,5
40,8032,88759,66,8524,4
41,26712,74890,67,8074,1
48,4548,5197,57,8181,5


In [75]:
candidates.loc[candidates['Price'].idxmin()]

Price              3608
Milage            12377
FuelEfficiency       69
Emissions          8129
Rating                1
Name: 62, dtype: int64

In [76]:
candidates.loc[candidates['Price'].idxmax()]

Price             34958
Milage            63472
FuelEfficiency       66
Emissions          8572
Rating                5
Name: 162, dtype: int64

In [77]:
budget = 12000
stars = 3
consider = candidates[ (candidates['Rating'] >= stars) & (candidates['Price'] <= budget) ]
consider

,Price,Milage,FuelEfficiency,Emissions,Rating
2,11052,40698,66,8279,3
10,7299,90345,69,8680,4
12,9329,29409,66,8364,4
20,5320,30628,65,8687,5
31,7181,19446,57,8081,5
40,8032,88759,66,8524,4
48,4548,5197,57,8181,5
70,3614,76627,68,8890,4
75,5395,99180,50,8333,4
77,4330,10807,38,8391,5


In [78]:
importance = { 'Milage': 0.3, 'FuelEfficiency': 0.5, 'Emissions': 0.2 }
sum(importance.values()) 

1.0

In [79]:
attr = importance.keys()
factor = [ importance[a] * ideal[a] for a in attr ]
factor

[0.3, -0.5, 0.2]

In [89]:
compromise = []
for i, data in consider.iterrows():
  combo = sum( [ f * d for f, d in zip(factor, data) ])
  compromise.append(combo)

cand = consider.copy()
l = 'Combination'
cand[l] = compromise
sorted = cand.sort_values(by = [l])
sorted.head(10) # top ten in terms of the lowest (best) combination score 

,Price,Milage,FuelEfficiency,Emissions,Rating,Combination
165,7248,129117,70,8323,5,-62370.1
107,6890,125624,59,8884,4,-60733.2
163,3860,107869,57,8911,5,-52765.1
75,5395,99180,50,8333,4,-47961.5
127,3619,95248,62,8002,3,-46525.9
10,7299,90345,69,8680,4,-42969.0
196,9752,91568,49,8062,3,-42848.6
40,8032,88759,66,8524,4,-41956.7
70,3614,76627,68,8890,4,-37215.7
185,5694,76257,48,8302,4,-36410.7
